In [3]:
# Global-Local Features Fusion for Lung Cancer Detection using DICOM Images

# Install required libraries
!pip install pydicom scikit-learn tensorflow xgboost opencv-python scikit-image seaborn matplotlib

import os
import pydicom
import numpy as np
import tensorflow as tf
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from skimage.filters import gabor
from skimage.feature import local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# Step 1: Load and preprocess DICOM images
def load_dicom_images(dicom_dir):
    """Load DICOM images from a directory."""
    dicom_files = [os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    images = []
    for file in dicom_files:
        dicom_data = pydicom.dcmread(file)
        image = dicom_data.pixel_array
        images.append(image)
    return np.array(images)

def preprocess_images(images):
    """Preprocess images by normalizing and denoising."""
    preprocessed_images = []
    for image in images:
        # Normalize pixel intensities
        normalized_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
        # Apply Gaussian blur for noise removal
        denoised_image = cv2.GaussianBlur(normalized_image, (5, 5), 0)
        preprocessed_images.append(denoised_image)
    return np.array(preprocessed_images)

# Step 2: Feature extraction
def extract_global_features(images):
    """Extract global features using a pre-trained DenseNet-201 model."""
    model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, pooling='avg')
    global_features = model.predict(images)
    return global_features

def extract_local_features(images):
    """Extract local features using Gabor filters and Local Binary Patterns (LBP)."""
    local_features = []
    for image in images:
        # Gabor filter
        gabor_features, _ = gabor(image, frequency=0.6)
        gabor_features = gabor_features.flatten()

        # Local Binary Patterns (LBP)
        lbp = local_binary_pattern(image, P=8, R=1, method="uniform")
        lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, 10), range=(0, 9))

        # Combine Gabor and LBP features
        local_features.append(np.concatenate([gabor_features, lbp_hist]))
    return np.array(local_features)

# Step 3: Feature fusion and PCA
def fuse_features(global_features, local_features):
    """Fuse global and local features and apply PCA for dimensionality reduction."""
    fused_features = np.concatenate([global_features, local_features], axis=1)
    pca = PCA(n_components=128)  # Adjust n_components as needed
    fused_features_pca = pca.fit_transform(fused_features)
    return fused_features_pca

# Step 4: Train classifiers
def train_classifiers(X_train, y_train):
    """Train multiple classifiers."""
    classifiers = {
        'LDA': LDA(),
        'GNB': GNB(),
        'KNN': KNN(n_neighbors=5),
        'SVM': SVC(kernel='linear', probability=True),
        'XGBoost': XGBClassifier()
    }
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
    return classifiers

# Step 5: Evaluate the model
def evaluate_model(y_true, y_pred):
    """Evaluate model performance using accuracy, recall, precision, and F1-score."""
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1': f1}

# Step 6: Cross-validation
def cross_validate_model(clf, X, y, cv=10):
    """Perform cross-validation and return mean accuracy."""
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    return scores.mean()

# Main pipeline
def main():
    # Load and preprocess DICOM images
    dicom_dir = r'G:\NBIA\VINOD_PhD_IKGPTUK\Lung_Cancer_Detection_MATLAB-Vinod\CNN_AlexNet_GoogleNet\dataset'
    images = load_dicom_images(dicom_dir)
    preprocessed_images = preprocess_images(images)

    # Extract features
    global_features = extract_global_features(preprocessed_images)
    local_features = extract_local_features(preprocessed_images)

    # Fuse features and apply PCA
    fused_features = fuse_features(global_features, local_features)

    # Train classifiers (assuming labels are available)
    X_train, y_train = fused_features, np.array([0, 1, 0, 1, 0])  # Replace with actual labels
    classifiers = train_classifiers(X_train, y_train)

    # Evaluate classifiers
    for name, clf in classifiers.items():
        y_pred = clf.predict(X_train)
        metrics = evaluate_model(y_train, y_pred)
        print(f"{name} Metrics:", metrics)

    # Cross-validation
    for name, clf in classifiers.items():
        cv_score = cross_validate_model(clf, X_train, y_train)
        print(f"{name} Cross-Validation Accuracy: {cv_score:.2f}")

    # Plot feature importance for XGBoost
    xgb = classifiers['XGBoost']
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(xgb.feature_importances_)), xgb.feature_importances_)
    plt.title("XGBoost Feature Importance")
    plt.xlabel("Feature Index")
    plt.ylabel("Importance")
    plt.show()

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\NBIA\\VINOD_PhD_IKGPTUK\\Lung_Cancer_Detection_MATLAB-Vinod\\CNN_AlexNet_GoogleNet\\dataset'